In [5]:
import os
import pandas as pd
from datetime import datetime


In [6]:
# Set paths
input_folder = 'Data1.0'
cheap_folder = os.path.join('Data2.0', 'Cheap')
expensive_folder = os.path.join('Data2.0', 'Expensive')
today_str = datetime.today().strftime('%Y-%m-%d')

# Create output directories if they don't exist
os.makedirs(cheap_folder, exist_ok=True)
os.makedirs(expensive_folder, exist_ok=True)

# Loop through each CSV file in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        try:
            df = pd.read_csv(input_path)

            # Ensure 'Purchase price' exists
            if 'Purchase price' in df.columns:
                # Convert to numeric, handle errors
                df['Purchase price'] = pd.to_numeric(df['Purchase price'], errors='coerce')

                # Separate cheap and expensive rows
                cheap_df = df[df['Purchase price'] < 2].copy()
                expensive_df = df[df['Purchase price'] >= 2].copy()

                # Function to enforce 'Language' column = 'en'
                def set_language(df_subset):
                    if 'Language' not in df_subset.columns:
                        df_subset['Language'] = 'en'
                    else:
                        df_subset['Language'] = df_subset['Language'].apply(lambda x: 'en' if str(x).strip().lower() != 'en' else x)
                    return df_subset

                # Apply language fix
                cheap_df = set_language(cheap_df)
                expensive_df = set_language(expensive_df)

                # Prepare filenames
                base_name = os.path.splitext(filename)[0]
                cheap_file = os.path.join(cheap_folder, f"{base_name}_{today_str}.csv")
                expensive_file = os.path.join(expensive_folder, f"{base_name}_{today_str}.csv")

                # Write CSVs if rows exist
                if not cheap_df.empty:
                    cheap_df.to_csv(cheap_file, index=False)
                    print(f"Cheap items written: {cheap_file}")
                if not expensive_df.empty:
                    expensive_df.to_csv(expensive_file, index=False)
                    print(f"Expensive items written: {expensive_file}")

            else:
                print(f"'Purchase price' column missing in: {filename}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

Cheap items written: Data2.0/Cheap/Moxfield-Blue_2025-06-10.csv
Expensive items written: Data2.0/Expensive/Moxfield-Blue_2025-06-10.csv
Cheap items written: Data2.0/Cheap/Moxfield-Black_2025-06-10.csv
Expensive items written: Data2.0/Expensive/Moxfield-Black_2025-06-10.csv
Cheap items written: Data2.0/Cheap/Moxfield-White_2025-06-10.csv
Expensive items written: Data2.0/Expensive/Moxfield-White_2025-06-10.csv
